In [1]:
import sqlite3
import numpy as np
import pandas as pd
from struct import unpack, calcsize

In [52]:
dbfile = 'testing.dltp'

In [53]:
query = """SELECT
        i.url AS "Image Path",
        l.id AS "Label ID",
        lc.name AS "Label Class",
        l.regiontype AS "Region Type",
        l.region AS "Region"
    FROM
        images i
    JOIN
        labels l ON i.id = l.image_id
    JOIN
        labelclasses lc ON l.labelclass_id = lc.id"""

In [54]:
con = sqlite3.connect(dbfile)
cur = con.cursor()
dataset = [a for a in cur.execute(query)]
con.close()

In [3]:
def decode_region(data):
    """Decode a binary polygon or mask and return it components"""
    payload = data[13:]

    if data[5:13] == b'\x00M\x00A\x00S\x00K':
        return _decode_mask(payload)

    if data[5:13] == b'\x00P\x00O\x00L\x00Y':
        return _decode_poly(payload)

    assert False, "Unable to decode this region"

def _decode_mask(payload):
    """Decodes the binary data into a list of pixels that form the pixelmask.
    Most likely format for the pixels is 'mask is on x-coordinates #0 until #1 on row #2"""
    # Create the struct mask, a list of unsigned long long
    mask = '>' + (len(payload) // 8) * 'Q'
    result = unpack(mask, payload)

    # Split into separate records
    final = [result[i*3+1:i*3+4] for i in range(len(result)//3)]

    assert len(final) == result[0], 'Number of pixels does not match sanity check'
    return "pixelmask", final

def _decode_poly(payload):
    """Decodes the binary data into a list of pixel coordinates that combine into a single polygon"""
    # Create the struct mask, a long list of doubles
    mask = '>Q' + (len(payload) // 8 - 1) * 'd'
    result = unpack(mask, payload)

    # Split into separate records
    final = [result[i*2+1:i*2+3] for i in range(len(result)//2)]

    assert len(final) == result[0], 'Number of pixels does not match sanity check'
    return "polygon", final


In [7]:
path = "/Users/tager/Developer/School/MachineLearning/Thema 2/Opdracht/logs/2022-04-21_13-39-09.506268_000029/2022-04-21_13-39-09.853778_detection_000.bin"

with open(path, "rb") as f:
    data = f.read()

decode_region(data)

AssertionError: Unable to decode this region

In [56]:
for record in dataset:
    print(decode_region(record[4]))

('pixelmask', [(0, 1, 0), (0, 1, 1)])
('pixelmask', [(0, 1, 0), (0, 1, 1), (0, 1, 2)])
('pixelmask', [(0, 2, 1), (0, 2, 2)])
('pixelmask', [(0, 1, 1), (1, 1, 2)])
('polygon', [(2.189655065536499, 3.6163792610168457), (1.9353448275862069, 2.461206896551724), (3.7543103448275863, 2.3448275862068964), (3.3836207389831543, 3.5862069129943848)])
